# Classifier Model

Create Path to Import *.py files (not needed in "index.ipynb")

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    sys.path.append(module_path+'/data')

#### sklearn libraries

In [2]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from codes.preprocess import *

#### Load clean train data.

In [44]:
df = pd.read_csv(module_path+'/data/cleaned_train_data.csv.zip')

In [45]:
X_train = df.drop(columns=['delinquency_bool'])
y_train = df.delinquency_bool

In [52]:
X_train[y_train==1].isna().sum()

id                           0
org_balance                  0
interest_rate_x              0
ltv                          0
score                       29
loan_purpose                 0
dti                         19
occupancy_type               0
property_type                0
reporting_period             0
interest_rate_y              0
upc_balance                  0
loan_age                     0
months_to_maturity           0
adj_months_to_maturity       0
msa                          0
delinquency_status           0
modification_flag            0
payment_amounts           1824
dtype: int64

In [53]:
X_train[y_train==0].isna().sum()

id                          0
org_balance                 0
interest_rate_x             0
ltv                         0
score                     725
loan_purpose                0
dti                       291
occupancy_type              0
property_type               0
reporting_period            0
interest_rate_y             0
upc_balance                 0
loan_age                    0
months_to_maturity          0
adj_months_to_maturity      0
msa                         0
delinquency_status          0
modification_flag           0
payment_amounts           729
dtype: int64

Testing pipe and grid with subset of 5000.

In [16]:
X_train = X_train[0:5000].fillna(0)
y_train = y_train[0:5000]

In [35]:
X_train

,Unnamed: 0,id,org_balance,interest_rate_x,ltv,score,loan_purpose,dti,occupancy_type,property_type,reporting_period,interest_rate_y,upc_balance,loan_age,months_to_maturity,adj_months_to_maturity,msa,delinquency_status,modification_flag,payment_amounts
0,599691,517830707527,79000,4.750,95,686.0,P,28.0,P,SF,2019-09-01,4.750,77002.38,14,346,343.0,38300,0,N,105.96
1,1545977,832752250469,167000,4.750,95,774.0,P,31.0,P,SF,2019-09-01,4.750,165058.56,10,350,350.0,22020,0,N,217.97
2,286339,707705221705,255000,4.500,75,689.5,C,24.0,P,SF,2019-09-01,4.500,248170.98,18,342,341.0,42660,0,N,416.74
3,628109,585358604766,208000,4.500,80,776.5,P,32.0,P,PU,2019-09-01,4.500,203781.73,15,345,345.0,41700,0,N,288.65
4,743694,858558980884,280000,4.625,80,819.0,P,21.0,P,SF,2019-08-01,4.625,202928.74,14,166,117.0,0,0,N,5623.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,651906,642529870927,453000,4.250,67,757.5,P,48.0,P,SF,2019-09-01,4.250,441421.27,18,342,342.0,41180,0,N,662.78
4996,418668,989349794537,267000,3.700,67,705.0,R,21.0,P,SF,2019-09-01,3.700,251102.05,21,219,219.0,11100,0,N,801.94
4997,1058893,616729132204,135000,5.250,71,644.0,R,40.0,P,SF,2019-09-01,5.250,132769.45,13,347,346.0,12940,0,N,163.89
4998,787742,962680216113,87000,4.750,90,797.0,P,21.0,P,CO,2019-09-01,4.750,85722.07,14,346,346.0,38060,0,N,115.63


In [15]:
y_train.value_counts()

0    4782
1     218
Name: delinquency_bool, dtype: int64

In [32]:
categ_cols = X_train.columns[X_train.dtypes==object].tolist()

In [33]:
numeric_cols = X_train.columns[X_train.dtypes!=object].tolist()

In [34]:
print(categ_cols,'\n',numeric_cols)

['loan_purpose', 'occupancy_type', 'property_type', 'reporting_period', 'modification_flag'] 
 ['Unnamed: 0', 'id', 'org_balance', 'interest_rate_x', 'ltv', 'score', 'dti', 'interest_rate_y', 'upc_balance', 'loan_age', 'months_to_maturity', 'adj_months_to_maturity', 'msa', 'delinquency_status', 'payment_amounts']


#### Build a base pipeline.

In [27]:
import time

In [ ]:
steps = dict(
    std_sc=StandardScaler(),
    dummies=OneHotEncoder(),
    estm=LogisticRegression()
)

In [30]:
pipe = Pipeline(steps=steps.items())

In [31]:
pipe.fit(X_train,y_train)

ValueError: could not convert string to float: 'P'